In [1]:
!wget "https://www.dropbox.com/scl/fi/0y8k4ykfrrcueiweubd2y/dataset.rar?rlkey=jpiywjldddul6pp1y7nv8fe3p&st=ht50ttsn&dl=0" -O dataset.rar

--2025-10-25 21:14:22--  https://www.dropbox.com/scl/fi/0y8k4ykfrrcueiweubd2y/dataset.rar?rlkey=jpiywjldddul6pp1y7nv8fe3p&st=ht50ttsn&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc45cf86f195be4f4ee170face9e.dl.dropboxusercontent.com/cd/0/inline/Cz6NCDxH_Ukeydhw4nF3E0pMVdojAjq1M_DQQgWB4PImz5f3ON9W1Pzs8KNEmtSu-r8ZhdfBNfI7g72HNuhrWNVGe6OeTuTb-TfzMWGVcwX5FQ5ZNwMxHSzS0x0KJPpTYmQ0RLlnPVbnL-IqmjGq1Bm2/file# [following]
--2025-10-25 21:14:23--  https://uc45cf86f195be4f4ee170face9e.dl.dropboxusercontent.com/cd/0/inline/Cz6NCDxH_Ukeydhw4nF3E0pMVdojAjq1M_DQQgWB4PImz5f3ON9W1Pzs8KNEmtSu-r8ZhdfBNfI7g72HNuhrWNVGe6OeTuTb-TfzMWGVcwX5FQ5ZNwMxHSzS0x0KJPpTYmQ0RLlnPVbnL-IqmjGq1Bm2/file
Resolving uc45cf86f195be4f4ee170face9e.dl.dropboxusercontent.com (uc45cf86f195be4f4ee170face9e.dl.dropboxusercontent.com)... 162.

In [2]:
!unrar x '/content/dataset.rar'


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/dataset.rar

Creating    dataset                                                   OK
Creating    dataset/content                                           OK
Creating    dataset/content/dataset                                   OK
Extracting  dataset/content/dataset/classes.txt                            0%  OK 
Creating    dataset/content/dataset/images                            OK
Extracting  dataset/content/dataset/images/frame_00000.jpg                 2%  OK 
Extracting  dataset/content/dataset/images/frame_00001.jpg                 5%  OK 
Extracting  dataset/content/dataset/images/frame_00002.jpg                 8%  OK 
Extracting  dataset/content/dataset/images/frame_00003.jpg                10%  OK 
Extracting  dataset/content/dataset/images/frame_00004.jpg                13%  OK 
Extracting  dataset/content/dataset/images

In [3]:
import torch, cv2, os, sys
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
!pip -q install ultralytics opencv-python numpy lap scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.2 MB/s eta 0:00:00


In [5]:
import cv2, time, numpy as np, os
from collections import deque, defaultdict
from ultralytics import YOLO
from IPython.display import clear_output
from google.colab.patches import cv2_imshow
import glob

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
def draw_corner_box(img, x1, y1, x2, y2, color_box=(0,255,0), color_corners=(255,0,0), t=2, corner=14):
    cv2.rectangle(img, (x1,y1), (x2,y2), color_box, t)
    cv2.line(img,(x1,y1),(x1+corner,y1),color_corners,t); cv2.line(img,(x1,y1),(x1,y1+corner),color_corners,t)
    cv2.line(img,(x2,y1),(x2-corner,y1),color_corners,t); cv2.line(img,(x2,y1),(x2,y1+corner),color_corners,t)
    cv2.line(img,(x1,y2),(x1+corner,y2),color_corners,t); cv2.line(img,(x1,y2),(x1,y2-corner),color_corners,t)
    cv2.line(img,(x2,y2),(x2-corner,y2),color_corners,t); cv2.line(img,(x2,y2),(x2,y2-corner),color_corners,t)

def yolo_txt_to_boxes(txt_path, img_w, img_h):
    """Читает YOLO .txt и возвращает список боксов [x1,y1,x2,y2] для class==0."""
    boxes = []
    if not os.path.exists(txt_path):
        return boxes
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, xc, yc, w, h = parts
            if int(float(cls)) != 0:
                continue
            xc, yc, w, h = map(float, (xc, yc, w, h))
            x1 = (xc - w/2.0) * img_w
            y1 = (yc - h/2.0) * img_h
            x2 = (xc + w/2.0) * img_w
            y2 = (yc + h/2.0) * img_h
            boxes.append([x1, y1, x2, y2])
    return boxes

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0.0, ix2-ix1), max(0.0, iy2-iy1)
    inter = iw*ih
    area_a = max(0.0, ax2-ax1)*max(0.0, ay2-ay1)
    area_b = max(0.0, bx2-bx1)*max(0.0, by2-by1)
    denom = area_a + area_b - inter + 1e-9
    return inter/denom

def greedy_match(pred_boxes, gt_boxes, iou_thr=0.5):
    """Жадное сопоставление предсказаний с GT по IoU, возвращает TP, FP, FN и пары индексов."""
    matched_gt = set()
    tp = 0
    matches = []
    for pi, pb in enumerate(pred_boxes):
        best_j, best_iou = -1, 0.0
        for gj, gb in enumerate(gt_boxes):
            if gj in matched_gt:
                continue
            iou = iou_xyxy(pb, gb)
            if iou > best_iou:
                best_iou, best_j = iou, gj
        if best_iou >= iou_thr and best_j >= 0:
            matched_gt.add(best_j)
            tp += 1
            matches.append((pi, best_j))
    fp = len(pred_boxes) - tp
    fn = len(gt_boxes) - tp
    return tp, fp, fn, matches

def run_images_and_metrics(
    images_dir="images",
    labels_dir="labels",
    output_dir="pred_vis",
    model_name="yolov8l.pt",
    conf=0.45,
    imgsz=896,
    limit=5,
    iou_match=0.5
):
    os.makedirs(output_dir, exist_ok=True)
    model = YOLO(model_name)

    exts = ("*.jpg","*.jpeg","*.png","*.bmp")
    img_list = []
    for ext in exts:
        img_list.extend(glob.glob(os.path.join(images_dir, ext)))
    img_list = sorted(img_list)[:limit]
    if not img_list:
        raise RuntimeError(f"В {images_dir} не найдено изображений.")

    total_tp = total_fp = total_fn = 0

    for img_path in img_list:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Пропуск: не удалось прочитать {img_path}")
            continue
        H, W = img.shape[:2]
        stem = os.path.splitext(os.path.basename(img_path))[0]
        gt_path = os.path.join(labels_dir, f"{stem}.txt")
        gt_boxes = yolo_txt_to_boxes(gt_path, W, H)

        res = model.predict(img, imgsz=imgsz, conf=conf, iou=0.5, classes=[0], verbose=False)[0]

        pred_boxes = []
        pred_scores = []
        if res.boxes is not None and res.boxes.xyxy is not None:
            xyxy = res.boxes.xyxy.cpu().numpy()
            confs = res.boxes.conf.cpu().numpy() if res.boxes.conf is not None else np.ones((xyxy.shape[0],), dtype=float)
            if getattr(res.boxes, "cls", None) is not None:
                cls = res.boxes.cls.cpu().numpy().astype(int)
                keep = (cls == 0)
                xyxy = xyxy[keep]; confs = confs[keep]
            order = np.argsort(-confs)
            xyxy = xyxy[order]; confs = confs[order]
            pred_boxes = xyxy.tolist()
            pred_scores = confs.tolist()

        tp, fp, fn, matches = greedy_match(pred_boxes, gt_boxes, iou_thr=iou_match)
        total_tp += tp; total_fp += fp; total_fn += fn

        vis = img.copy()
        for x1,y1,x2,y2 in gt_boxes:
            draw_corner_box(vis, int(x1),int(y1),int(x2),int(y2), color_box=(255,0,0), color_corners=(255,0,0), t=2, corner=10)
        for k,(x1,y1,x2,y2) in enumerate(pred_boxes):
            draw_corner_box(vis, int(x1),int(y1),int(x2),int(y2), color_box=(0,255,0), color_corners=(0,255,0), t=2, corner=10)
            if k < len(pred_scores):
                s = pred_scores[k]
                label = f"{s*100:.1f}%"
                (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
                cv2.rectangle(vis, (int(x1), int(y1)-th-6), (int(x1)+tw+6, int(y1)), (0,0,0), -1)
                cv2.putText(vis, label, (int(x1)+3, int(y1)-4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

        cv2.putText(vis, f"TP:{tp} FP:{fp} FN:{fn}", (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(vis, f"TP:{tp} FP:{fp} FN:{fn}", (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)

        out_path = os.path.join(output_dir, f"{stem}_pred.jpg")
        cv2.imwrite(out_path, vis)

    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    recall    = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    f1        = 2*precision*recall / (precision+recall) if (precision+recall) > 0 else 0.0

    print(f"[Aggregate @IoU={iou_match:.2f}]  TP={total_tp}  FP={total_fp}  FN={total_fn}")
    print(f"Precision={precision:.4f}  Recall={recall:.4f}  F1={f1:.4f}")

    return {"TP": total_tp, "FP": total_fp, "FN": total_fn,
            "precision": precision, "recall": recall, "f1": f1}

In [7]:
for conf in [0.05, 0.10, 0.15, 0.20, 0.30, 0.45, 0.5, 0.55, 0.6, 0.7]:
    m = run_images_and_metrics(
        images_dir="/content/dataset/content/dataset/images",
        labels_dir="/content/dataset/content/dataset/labels_sync",
        output_dir="pred_vis",
        model_name="yolov8l.pt",
        conf=conf,
        imgsz=896,
        limit=40,
        iou_match=0.5
    )
    print(f"conf={conf:.2f} → P={m['precision']:.3f} R={m['recall']:.3f} F1={m['f1']:.3f}")

[Aggregate @IoU=0.50]  TP=499  FP=713  FN=287
Precision=0.4117  Recall=0.6349  F1=0.4995
conf=0.05 → P=0.412 R=0.635 F1=0.499
[Aggregate @IoU=0.50]  TP=495  FP=430  FN=291
Precision=0.5351  Recall=0.6298  F1=0.5786
conf=0.10 → P=0.535 R=0.630 F1=0.579
[Aggregate @IoU=0.50]  TP=492  FP=253  FN=294
Precision=0.6604  Recall=0.6260  F1=0.6427
conf=0.15 → P=0.660 R=0.626 F1=0.643
[Aggregate @IoU=0.50]  TP=488  FP=153  FN=298
Precision=0.7613  Recall=0.6209  F1=0.6840
conf=0.20 → P=0.761 R=0.621 F1=0.684
[Aggregate @IoU=0.50]  TP=472  FP=83  FN=314
Precision=0.8505  Recall=0.6005  F1=0.7040
conf=0.30 → P=0.850 R=0.601 F1=0.704
[Aggregate @IoU=0.50]  TP=406  FP=33  FN=380
Precision=0.9248  Recall=0.5165  F1=0.6629
conf=0.45 → P=0.925 R=0.517 F1=0.663
[Aggregate @IoU=0.50]  TP=391  FP=25  FN=395
Precision=0.9399  Recall=0.4975  F1=0.6506
conf=0.50 → P=0.940 R=0.497 F1=0.651
[Aggregate @IoU=0.50]  TP=368  FP=19  FN=418
Precision=0.9509  Recall=0.4682  F1=0.6275
conf=0.55 → P=0.951 R=0.468 F1=0.